In [5]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from multiprocessing.pool import ThreadPool
from multiprocessing import Process, Manager, Pool
from shared_funcs import gather_stats, merge_add_dict

In [6]:
chunks = pd.read_json('../data/arxiv-metadata-oai-snapshot.json', lines=True, chunksize=50000)

In [8]:
# manager = Manager()

# author_dict = manager.dict()
# cat_dict = manager.dict()
# year_dict = manager.dict()
author_dict = {}
cat_dict = {}

counter = 0
for chunk in chunks:
    df = chunk
    intermediate_authors_stats, intermediate_cat_stats = gather_stats(df)
    author_dict = merge_add_dict(author_dict, intermediate_authors_stats)
    cat_dict = merge_add_dict(cat_dict, intermediate_cat_stats)
    
    counter += len(df)
    print("processed:", counter)
    print("aut stats => ", len(author_dict))
    print("cat stats => ", len(cat_dict))
    print()
    # break

processed: 50000
aut stats =>  134472
cat stats =>  4029

processed: 100000
aut stats =>  226551
cat stats =>  6706

processed: 150000
aut stats =>  305563
cat stats =>  9574

processed: 200000
aut stats =>  376949
cat stats =>  12487

processed: 250000
aut stats =>  445084
cat stats =>  15202

processed: 300000
aut stats =>  511322
cat stats =>  17627

processed: 350000
aut stats =>  575602
cat stats =>  20039

processed: 400000
aut stats =>  642347
cat stats =>  22014

processed: 450000
aut stats =>  706180
cat stats =>  23840

processed: 500000
aut stats =>  768536
cat stats =>  25550

processed: 550000
aut stats =>  833687
cat stats =>  27308

processed: 600000
aut stats =>  894438
cat stats =>  29021

processed: 650000
aut stats =>  955159
cat stats =>  30651

processed: 700000
aut stats =>  1016144
cat stats =>  32237

processed: 750000
aut stats =>  1078456
cat stats =>  33795

processed: 800000
aut stats =>  1139613
cat stats =>  35396

processed: 850000
aut stats =>  1203130
c

In [9]:
def sort_dict(dict_data, byval=True, reverse=True):
    if byval: idx = 1
    else: idx = 0
    return dict(sorted(dict_data.items(), key=lambda item: item[idx], reverse=reverse))

In [ ]:
# sort_dict(author_dict)

# sort_dict(cat_dict)

In [ ]:
# # Convert dictionary to JSON formatted string
# json_data = json.dumps(author_dict, indent=4)

# # Writing JSON data to a file
# with open('../data/author_stats.json', 'w') as file:
#     file.write(json_data)